## 영화 5개 받기

In [24]:
print("좋아하는 영화 5개를 입력해주세요(구분은 ,로): ")
movie_list = input().split(",")
print(movie_list)

좋아하는 영화 5개를 입력해주세요(구분은 ,로): 
명량, 기생충, 극한직업, 패신저스, 메이즈러너
['명량', ' 기생충', ' 극한직업', ' 패신저스', ' 메이즈러너']


## 영화 정보

In [25]:
import requests
import json

In [26]:
movie_cd_list = []
director_list = []
selected_movie_list = []

for movie in movie_list:
    movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=e3a5ce5b387559b21c721bb76a8024a1&movieNm='+movie
    res = requests.get(movie_cd_url)
    text = res.text
    d = json.loads(text)
    if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
        item_year = input(movie+"의 개봉연도를 입력하세요: ")   # 개봉연도로 구분
        for item in d['movieListResult']['movieList']:
            if item_year == item['openDt'][:4]:
                # director_list = director['peopleNm']
                movie_cd_list.append([item['movieCd'],
                                      item['movieNm'],
                                      item['genreAlt'],
                                      item['openDt'][:4]])
                selected_movie_list.append(item['movieNm'])
    if d['movieListResult']['totCnt'] == 1:
        for item in d['movieListResult']['movieList']:
            movie_cd_list.append([item['movieCd'],
                                  item['movieNm'],
                                  item['genreAlt'],
                                  item['openDt'][:4]])
            selected_movie_list.append(item['movieNm'])

print(movie_cd_list)
print(selected_movie_list)

명량의 개봉연도를 입력하세요: 2014
 기생충의 개봉연도를 입력하세요: 2019
 극한직업의 개봉연도를 입력하세요: 2019
 패신저스의 개봉연도를 입력하세요: 2017
 메이즈러너의 개봉연도를 입력하세요: 2014
[['20129370', '명량', '사극,액션', '2014'], ['20183782', '기생충', '드라마', '2019'], ['20198543', '파라노말 기생충', '멜로/로맨스', '2019'], ['20182530', '극한직업', '코미디', '2019'], ['20161084', '패신저스', '어드벤처,SF', '2017'], ['20140431', '메이즈 러너', '액션,미스터리,SF,스릴러', '2014']]
['명량', '기생충', '파라노말 기생충', '극한직업', '패신저스', '메이즈 러너']


In [27]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','genreAlt', 'openYr']
movie_cdlist_df

,movieCd,movieNm,genreAlt,openYr
0,20129370,명량,"사극,액션",2014
1,20183782,기생충,드라마,2019
2,20198543,파라노말 기생충,멜로/로맨스,2019
3,20182530,극한직업,코미디,2019
4,20161084,패신저스,"어드벤처,SF",2017
5,20140431,메이즈 러너,"액션,미스터리,SF,스릴러",2014


## 영화코드로 정보 추출

In [28]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
movie_genre_list = [] 

actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
genre_list = []              # 5개 영화의 장르를 모은 것

movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
    movie_director_list.clear()
    movie_cast_list.clear()
    movie_mainactor_list.clear()
    movie_genre_list.clear()
    actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=e3a5ce5b387559b21c721bb76a8024a1&movieCd='+movieCd
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for direc in d['movieInfoResult']['movieInfo']['directors']:
        movie_director_list.append(direc['peopleNm'])
        direct_list+=movie_director_list

    for cast in d['movieInfoResult']['movieInfo']['actors']:
        if len(movie_mainactor_list) < 3:
            movie_mainactor_list.append(cast['peopleNm'])
        if len(movie_cast_list) <= 9:
            movie_cast_list.append(cast['peopleNm'])
            
    for genr in d['movieInfoResult']['movieInfo']['genres']:
        movie_genre_list.append(genr['genreNm'])
  
    #print(movie_cast_list)
    actor_list += movie_cast_list
    mainactor_list += movie_mainactor_list
    genre_list += movie_genre_list

    j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)
print(genre_list)

['김한민', '봉준호', '나츠메 타이치로', '이병헌', '모튼 틸덤', '웨스 볼']
['최민식', '류승룡', '조진웅', '김명곤', '진구', '이정현', '권율', '노민우', '이승준', '오타니 료헤이', '송강호', '이선균', '조여정', '최우식', '박소담', '이정은', '장혜진', '박명훈', '정지소', '정현준', '류승룡', '이하늬', '진선규', '이동휘', '공명', '양현민', '장진희', '한준우', '김명준', '나철', '크리스 프랫', '제니퍼 로렌스', '마이클 쉰', '앤디 가르시아', '로렌스 피쉬번', '딜런 오브라이언', '카야 스코델라리오', '윌 폴터', '토마스 생스터', '이기홍']
['최민식', '류승룡', '조진웅', '송강호', '이선균', '조여정', '류승룡', '이하늬', '진선규', '크리스 프랫', '제니퍼 로렌스', '마이클 쉰', '딜런 오브라이언', '카야 스코델라리오', '윌 폴터']
['사극', '액션', '드라마', '멜로/로맨스', '코미디', '어드벤처', 'SF', '액션', '미스터리', 'SF', '스릴러']


## 배우 등장 횟수

In [29]:
actor_dict = dict()
for i in actor_list:
    if i not in actor_dict:
        actor_dict[i]=1
    else:
        actor_dict[i]=actor_dict[i]+1

actor_dict

{'최민식': 1,
 '류승룡': 2,
 '조진웅': 1,
 '김명곤': 1,
 '진구': 1,
 '이정현': 1,
 '권율': 1,
 '노민우': 1,
 '이승준': 1,
 '오타니 료헤이': 1,
 '송강호': 1,
 '이선균': 1,
 '조여정': 1,
 '최우식': 1,
 '박소담': 1,
 '이정은': 1,
 '장혜진': 1,
 '박명훈': 1,
 '정지소': 1,
 '정현준': 1,
 '이하늬': 1,
 '진선규': 1,
 '이동휘': 1,
 '공명': 1,
 '양현민': 1,
 '장진희': 1,
 '한준우': 1,
 '김명준': 1,
 '나철': 1,
 '크리스 프랫': 1,
 '제니퍼 로렌스': 1,
 '마이클 쉰': 1,
 '앤디 가르시아': 1,
 '로렌스 피쉬번': 1,
 '딜런 오브라이언': 1,
 '카야 스코델라리오': 1,
 '윌 폴터': 1,
 '토마스 생스터': 1,
 '이기홍': 1}

## 주연배우(맨 앞 3사람)

In [30]:
mainactor_dict = dict()
for i in mainactor_list:
    if i not in mainactor_dict:
        mainactor_dict[i]=1
    else:
        mainactor_dict[i]=mainactor_dict[i]+1

mainactor_dict

{'최민식': 1,
 '류승룡': 2,
 '조진웅': 1,
 '송강호': 1,
 '이선균': 1,
 '조여정': 1,
 '이하늬': 1,
 '진선규': 1,
 '크리스 프랫': 1,
 '제니퍼 로렌스': 1,
 '마이클 쉰': 1,
 '딜런 오브라이언': 1,
 '카야 스코델라리오': 1,
 '윌 폴터': 1}

## 추천배우 5명

- 2번 등장 배우 우선
- 빈자리는 랜덤

In [31]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
    if value > 1:
        prefer_actor_list.append(key)
if len(prefer_actor_list)!=5:
    while True:
        actor = random.choice(mainactor_list)
        if actor not in prefer_actor_list:
            prefer_actor_list.append(actor)
        if len(prefer_actor_list)==5:
            break

prefer_actor_list

['류승룡', '딜런 오브라이언', '제니퍼 로렌스', '조여정', '카야 스코델라리오']

## 배우 추천 코드

In [32]:
actor_code = []
filmo_list = []
final_filmo_list = []

for prefer_actor in prefer_actor_list:
    actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key=e3a5ce5b387559b21c721bb76a8024a1'+'&peopleNm='+prefer_actor
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for actor in d['peopleListResult']['peopleList']:
        final_filmo_list.clear()
        if actor['repRoleNm'] == '배우' and actor['peopleNmEn'] != "":
            filmo_list = actor['filmoNames'].split("|")
            for filmo in filmo_list:
                if filmo not in selected_movie_list:
                    final_filmo_list.append(filmo)
            actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'], actor['repRoleNm'], final_filmo_list[:5]])

In [33]:
print(selected_movie_list)
print(filmo_list)
print(final_filmo_list)

['명량', '기생충', '파라노말 기생충', '극한직업', '패신저스', '메이즈 러너']
['크롤', '메이즈 러너: 데스 큐어', '캐리비안의 해적: 죽은 자는 말이 없다', '메이즈 러너: 스코치 트라이얼', '트루스 어바웃 엠마누엘', '메이즈 러너', '나우이즈굿 ', '폭풍의 언덕']
['크롤', '메이즈 러너: 데스 큐어', '캐리비안의 해적: 죽은 자는 말이 없다', '메이즈 러너: 스코치 트라이얼', '트루스 어바웃 엠마누엘', '나우이즈굿 ', '폭풍의 언덕']


In [34]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm', 'filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10019065,류승룡,RYU Seung-ryong,배우,"[비광, 정가네 목장(가제), 입술은 안돼요(가제), 자산어보, 인생은 아름다워]"
1,10019472,딜런 오브라이언,Dylan O'Brien,배우,"[플래시백, 메이즈 러너: 데스 큐어, 어쌔신: 더 비기닝, 딥워터 호라이즌, 메이..."
2,10062627,제니퍼 로렌스,Jennifer Lawrence,배우,"[엑스맨: 다크 피닉스, 레드 스패로, 마더!, 엑스맨: 아포칼립스, 조이]"
3,10066680,조여정,CHO Yeo-jeong,배우,"[워킹걸, 인간중독, 표적, 후궁 : 제왕의 첩, 방자전]"
4,10073877,카야 스코델라리오,Kaya Scodelario,배우,"[크롤, 메이즈 러너: 데스 큐어, 캐리비안의 해적: 죽은 자는 말이 없다, 메이즈..."


## 장르 등장 횟수 

In [35]:
genre_dict = dict()
for i in genre_list:
    if i not in genre_dict:
        genre_dict[i]=1
    else:
        genre_dict[i]=genre_dict[i]+1

genre_dict

{'사극': 1,
 '액션': 2,
 '드라마': 1,
 '멜로/로맨스': 1,
 '코미디': 1,
 '어드벤처': 1,
 'SF': 2,
 '미스터리': 1,
 '스릴러': 1}

In [36]:
genre_df = pd.DataFrame(genre_dict.items())
genre_df.columns=['genres', 'number']
genre_df

,genres,number
0,사극,1
1,액션,2
2,드라마,1
3,멜로/로맨스,1
4,코미디,1
5,어드벤처,1
6,SF,2
7,미스터리,1
8,스릴러,1


In [37]:
gen=dict()
gen=genre_df['genres']
gg=gen.values
gg

array(['사극', '액션', '드라마', '멜로/로맨스', '코미디', '어드벤처', 'SF', '미스터리', '스릴러'],
      dtype=object)

In [38]:
recom_genre=[]
recom_genre=genre_df[genre_df['number']>1]
recom_genre_list=recom_genre['genres']
recom_genre_list

1    액션
6    SF
Name: genres, dtype: object

In [39]:
recom_genre_list=recom_genre_list.values
recom_genre_list

array(['액션', 'SF'], dtype=object)

## 추려진 영화 장르 검색

In [40]:
genre_search=[]
movieID=[]
prdY=[]
odt=[]
movieN=[]
moviepeople=[]

for i in range(len(final_filmo_list)):
    genre_search_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=e3a5ce5b387559b21c721bb76a8024a1'+'&movieNm='+final_filmo_list[i]
    res = requests.get(genre_search_url)
    text = res.text
    d = json.loads(text)

    if d['movieListResult']['totCnt'] > 1:
        for item in d['movieListResult']['movieList']:
            movieID.append(item['movieCd'])

    if d['movieListResult']['totCnt'] == 1:        
        for item in d['movieListResult']['movieList']:
            genre_search.append([item['movieNm'], item['genreAlt']])
            
for i in range(len(movieID)):
    id_search_url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=e3a5ce5b387559b21c721bb76a8024a1'+'&movieCd='+movieID[i]
    res = requests.get(id_search_url)
    text = res.text
    d = json.loads(text)
    
    dd=d['movieInfoResult']['movieInfo']
    for item in d['movieInfoResult']['movieInfo']['actors']:
        moviepeople=item['peopleNm']
        if moviepeople in prefer_actor_list:
            movieN.append(dd['movieNm'])
            prdY.append(dd['prdtYear'])
            odt.append(dd['openDt'])

for i in range(len(movieN)):
    duplit_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=e3a5ce5b387559b21c721bb76a8024a1'+'&movieNm='+movieN[i]
    res = requests.get(duplit_url)
    text = res.text
    d = json.loads(text)
    
    for item in d['movieListResult']['movieList']:
        if item['openDt']==odt[i]:
            genre_search.append([item['movieNm'], item['genreAlt']])
            break
        if item['prdtYear']==prdY[i]:
            genre_search.append([item['movieNm'], item['genreAlt']])

print(genre_search)

[['메이즈 러너: 데스 큐어', '액션,SF,스릴러'], ['캐리비안의 해적: 죽은 자는 말이 없다', '액션,어드벤처,판타지,코미디'], ['메이즈 러너: 스코치 트라이얼', '액션,SF,스릴러'], ['트루스 어바웃 엠마누엘', '드라마,스릴러'], ['나우이즈굿 ', '드라마,멜로/로맨스'], ['크롤', '액션,스릴러'], ['폭풍의 언덕', '드라마,멜로/로맨스']]


In [41]:
pre_recom_pd=pd.DataFrame(genre_search)
pre_recom_pd.columns=['movieNm', 'genres']
pre_recom_pd

,movieNm,genres
0,메이즈 러너: 데스 큐어,"액션,SF,스릴러"
1,캐리비안의 해적: 죽은 자는 말이 없다,"액션,어드벤처,판타지,코미디"
2,메이즈 러너: 스코치 트라이얼,"액션,SF,스릴러"
3,트루스 어바웃 엠마누엘,"드라마,스릴러"
4,나우이즈굿,"드라마,멜로/로맨스"
5,크롤,"액션,스릴러"
6,폭풍의 언덕,"드라마,멜로/로맨스"


## 추천 장르로 영화 추출

In [63]:
recom_genre=dict()
real=dict()
for i in range(len(recom_genre_list)):
    recom_genre=pre_recom_pd[pre_recom_pd['genres'].str.contains(recom_genre_list[i])]['movieNm']
    real.update(recom_genre)
real

{0: '메이즈 러너: 데스 큐어',
 1: '캐리비안의 해적: 죽은 자는 말이 없다',
 2: '메이즈 러너: 스코치 트라이얼',
 5: '크롤'}

In [69]:
recommendation_list=[]
for key, value in real.items():
    recommendation_list.append(value)
recommendation_list

['메이즈 러너: 데스 큐어', '캐리비안의 해적: 죽은 자는 말이 없다', '메이즈 러너: 스코치 트라이얼', '크롤']

## 영화추천

In [70]:
if len(recommendation)<5:
    recommendation=random.sample(recommendation, len(recommendation))
else:
    recommendation=random.sample(recommendation, 5)
recommendation

['크롤', '메이즈 러너: 데스 큐어', '메이즈 러너: 스코치 트라이얼', '캐리비안의 해적: 죽은 자는 말이 없다']